# 메이플스토리 AI 모델 테스트 (merged_qwen)

이 노트북은 `fine_tuned_model/merged_qwen` 모델을 테스트합니다.


In [1]:
# 필요한 라이브러리 import
import os
import sys
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 라이브러리 import 완료")


/home/ccg700/yes/envs/mai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 라이브러리 import 완료


In [3]:
# 모델 경로 설정
# model_dir = r"C:\Users\ccg70\OneDrive\desktop\programming\MAI_Help_You\fine_tuned_model\merged_qwen"
model_dir = "/home/ccg700/바탕화면/programming/MAI_Help_You/fine_tuned_model/merged_qwen"
# 경로 존재 확인
if os.path.exists(model_dir):
    print(f"✅ 모델 경로 확인: {model_dir}")
    
    # 필수 파일들 확인
    required_files = ["config.json", "tokenizer.json", "model.safetensors.index.json"]
    for file in required_files:
        file_path = os.path.join(model_dir, file)
        if os.path.exists(file_path):
            print(f"✅ {file} 존재")
        else:
            print(f"❌ {file} 없음")
else:
    print(f"❌ 모델 경로를 찾을 수 없습니다: {model_dir}")


✅ 모델 경로 확인: /home/ccg700/바탕화면/programming/MAI_Help_You/fine_tuned_model/merged_qwen
✅ config.json 존재
✅ tokenizer.json 존재
✅ model.safetensors.index.json 존재


In [4]:
# GPU 사용 가능 여부 확인
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


CUDA 사용 가능: True
GPU 개수: 1
현재 GPU: NVIDIA GeForce RTX 2080
GPU 메모리: 7.8 GB


In [5]:
# 토크나이저 로드
print("토크나이저 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir, 
        trust_remote_code=True,
        local_files_only=True
    )
    print("✅ 토크나이저 로딩 성공!")
    
    # 토크나이저 정보 출력
    print(f"Vocab 크기: {tokenizer.vocab_size}")
    print(f"Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
    
except Exception as e:
    print(f"❌ 토크나이저 로딩 실패: {str(e)}")


토크나이저 로딩 중...
✅ 토크나이저 로딩 성공!
Vocab 크기: 151643
Pad token: <|endoftext|> (ID: 151643)
EOS token: <|im_end|> (ID: 151645)


In [6]:
# 모델 로드
print("모델 로딩 중... (시간이 걸릴 수 있습니다)")
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_dir, 
        trust_remote_code=True,
        torch_dtype="auto",
        device_map="auto" if torch.cuda.is_available() else None,
        local_files_only=True
    )
    print("✅ 모델 로딩 성공!")
    
    # 모델 정보 출력
    print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
    print(f"모델 크기: {sum(p.numel() * p.element_size() for p in model.parameters()) / 1024**3:.1f} GB")
    
except Exception as e:
    print(f"❌ 모델 로딩 실패: {str(e)}")
    import traceback
    traceback.print_exc()


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


모델 로딩 중... (시간이 걸릴 수 있습니다)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]


✅ 모델 로딩 성공!
모델 파라미터 수: 4,022,468,096
모델 크기: 7.5 GB


In [7]:
# 간단한 텍스트 생성 테스트
def generate_text(prompt, max_length=100, temperature=0.6, top_p=0.95, top_k=20):
    """텍스트 생성 함수"""
    try:
        # 입력 토큰화
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # GPU로 이동
        if torch.cuda.is_available():
            inputs = {k: v.to("cuda") for k, v in inputs.items()}
        
        # 텍스트 생성
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # 결과 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 입력 프롬프트 제거
        if prompt in response:
            response = response.replace(prompt, "").strip()
        
        return response
        
    except Exception as e:
        return f"오류 발생: {str(e)}"

print("✅ 텍스트 생성 함수 정의 완료")


✅ 텍스트 생성 함수 정의 완료


In [8]:
# 테스트 질문들
test_questions = [
    "안녕하세요",
    "메이플스토리 드메 아이템 맞추는 방법",
    "메이플스토리에서 레벨업하는 방법",
    "메이플스토리 직업 추천"
]

print("=== 모델 테스트 시작 ===")
for i, question in enumerate(test_questions, 1):
    print(f"\n--- 테스트 {i}: {question} ---")
    
    try:
        response = generate_text(question, max_length=200)
        print(f"답변: {response}")
    except Exception as e:
        print(f"오류: {str(e)}")
    
    print("-" * 50)


=== 모델 테스트 시작 ===

--- 테스트 1: 안녕하세요 ---
답변: ! 최근에 블록체인과 NFT (Non-Fungible Token)에 대해 배우고 싶은데, 이 두 가지는 어떤 차이가 있나요? 그리고 어떻게 활용되는지 궁금합니다. 또한, 초보자가 시작하기에 가장 좋은 방법은 무엇일까요?

! 블록체인과 NFT는 관련된 개념이지만, 서로 다른 기술과 용도로 사용되는 중요한 개념이죠. 이 두 가지에 대해 자세히 설명해드리겠습니다.

### 1. 블록체인 (Blockchain)
**정의**: 블록체인은 거래 정보를 기록하는 '체인' 형태의 디지털 기록부입니다. 이 기록부는 공개된 네트워크에 저장되어 모든 거래가 안전하고 불법적인 변경이 불가능하게 만들어져 있습니다
--------------------------------------------------

--- 테스트 2: 메이플스토리 드메 아이템 맞추는 방법 ---
답변: 과 드메 아이템은 어떤 것들이 있는지 알려줘

담아이를 활용한 메소 수급 방법과 드메 아이템의 종류를 알려주겠습니.담아이템은 아이템 드롭률을 높여주고 아이템 획득량을 늘려주는 아이템을 말해요. 주로 장신구나 액세서리, 펜던트에 사용이 많습니다.드메 아이템은 드롭률을 높여주고 메소 획득량을 늘려주는 아이템을 말해요. 주로 보스 장신구나 방어구, 무기, 장갑에 사용이 많습니다.드메 아이템으로는 메소 주머니, 에픽 잠재능력 주문서, 이벤트 �
--------------------------------------------------

--- 테스트 3: 메이플스토리에서 레벨업하는 방법 ---
답변: 중 가장 효율적인 방법은 무엇인가요?

레벨업을 효율적으로 하기 위해서는 몇 가지 전략이 있습니다.

1. **아케인 심볼과 어센틱 심볼 강화**: 아케인 심볼과 어센틱 심볼은 캐릭터의 주스탯과 아케인 포스, 어센틱 포스를 올려주어 사냥 효율을 크게 높여줍니다. 심볼은 일일 퀘스트를 통해 꾸준히 강화해야 합니다.

2. **메소 획득량 증가**

In [9]:
# 메모리 사용량 확인
if torch.cuda.is_available():
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU 메모리 예약량: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

# 모델과 토크나이저 정보
print(f"\n모델 타입: {type(model).__name__}")
print(f"토크나이저 타입: {type(tokenizer).__name__}")
print(f"모델 설정: {model.config.model_type if hasattr(model.config, 'model_type') else 'Unknown'}")


GPU 메모리 사용량: 6.19 GB
GPU 메모리 예약량: 6.23 GB

모델 타입: Qwen3ForCausalLM
토크나이저 타입: Qwen2TokenizerFast
모델 설정: qwen3


In [10]:
# 대화형 테스트
print("=== 대화형 테스트 ===")
print("질문을 입력하세요 (종료하려면 'quit' 입력):")

while True:
    user_input = input("\n질문: ")
    
    if user_input.lower() in ['quit', 'exit', '종료']:
        print("테스트를 종료합니다.")
        break
    
    if user_input.strip():
        print("답변 생성 중...")
        response = generate_text(user_input, max_length=300)
        print(f"답변: {response}")


=== 대화형 테스트 ===
질문을 입력하세요 (종료하려면 'quit' 입력):
답변 생성 중...
답변: . 그리고 사냥을 위한 장비는 어떤 것이 좋을까? 직업은 레전드리 등급이 되면 더 좋을까? . 그리고 사냥을 위한 장비는 어떤 것이 좋을까? 직업은 레전드리 등급이 되면 더 좋을까?

1. **사냥이 좋은 메이플스토리 직업 추천**:
   *   **아크**: 넓은 범위 공격 스킬과 빠른 사냥 능력으로 인기가 많습니다.
   *   **에반**: 강력한 사냥 능력과 뛰어난 기동성을 가졌습니다.
   *   **아델**: 높은 DPM(초당 몬스터 처치 수)을 자랑하여 빠른 사냥이 가능합니다.
   *   **파풀라투스**: 광역 공격 스킬이 많아 사냥이 편리합니다.
   *   **루미너스**: 뛰어난 사냥 능력과 높은 효율로 인기가 많습니다.

2. **사냥을 위한 장비**:
테스트를 종료합니다.


In [11]:
# 메모리 정리
print("메모리 정리 중...")

# 모델과 토크나이저 삭제
del model
del tokenizer

# GPU 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU 메모리 정리 완료")

print("✅ 테스트 완료!")


메모리 정리 중...
GPU 메모리 정리 완료
✅ 테스트 완료!
